In [1]:
# Imports
import os
import sys
from pyspark.sql import SparkSession
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.master("local[1]") \
    .appName("Sales").getOrCreate()

In [18]:
salesDF = spark.read.options(header="True",delimiter=",",inferSchema = "True").format("csv").load("sales.csv")

In [19]:
salesDF.printSchema()
salesDF.show()

root
 |-- SalesOrderNumber: string (nullable = true)
 |-- SalesOrderLineNumber: integer (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- TaxAmount: double (nullable = true)

+----------------+--------------------+----------+-----------------+--------------------+--------------------+--------+---------+---------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|     CustomerName|        EmailAddress|                Item|Quantity|UnitPrice|TaxAmount|
+----------------+--------------------+----------+-----------------+--------------------+--------------------+--------+---------+---------+
|         SO43701|                   1|2019-07-01|      Christy Zhu|christy12@adventu...|Mountain-100 Silv...|       1|  3399.99| 271.9992|
|         SO43704|                   1|2019-07-

In [23]:
salesDF.createOrReplaceTempView("sales")
spark.sql("select * from sales").show()

+----------------+--------------------+----------+-----------------+--------------------+--------------------+--------+---------+---------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|     CustomerName|        EmailAddress|                Item|Quantity|UnitPrice|TaxAmount|
+----------------+--------------------+----------+-----------------+--------------------+--------------------+--------+---------+---------+
|         SO43701|                   1|2019-07-01|      Christy Zhu|christy12@adventu...|Mountain-100 Silv...|       1|  3399.99| 271.9992|
|         SO43704|                   1|2019-07-01|       Julio Ruiz|julio1@adventure-...|Mountain-100 Blac...|       1|  3374.99| 269.9992|
|         SO43705|                   1|2019-07-01|        Curtis Lu|curtis9@adventure...|Mountain-100 Silv...|       1|  3399.99| 271.9992|
|         SO43700|                   1|2019-07-01|     Ruben Prasad|ruben10@adventure...|  Road-650 Black, 62|       1| 699.0982|  55.9279|
|         SO43703|  

In [34]:
salesDF.repartition(2)\
.write.options(header="True")\
.option("maxRecordsPerFile", 5)\
.mode("overwrite")\
.partitionBy("OrderDate")\
.csv("sales")

In [35]:
salesDF.createOrReplaceTempView("salestable")
spark.sql("select Item,OrderDate from salestable where OrderDate = '2019-07-01'").show(truncate=False)

+-----------------------+----------+
|Item                   |OrderDate |
+-----------------------+----------+
|Mountain-100 Silver, 44|2019-07-01|
|Mountain-100 Black, 48 |2019-07-01|
|Mountain-100 Silver, 38|2019-07-01|
|Road-650 Black, 62     |2019-07-01|
|Road-150 Red, 62       |2019-07-01|
|Road-150 Red, 62       |2019-07-01|
|Mountain-100 Silver, 44|2019-07-01|
|Road-150 Red, 44       |2019-07-01|
|Mountain-100 Silver, 44|2019-07-01|
+-----------------------+----------+



In [42]:
print(salesDF.sample(0.1).collect())

[Row(SalesOrderNumber='SO43700', SalesOrderLineNumber=1, OrderDate=datetime.date(2019, 7, 1), CustomerName='Ruben Prasad', EmailAddress='ruben10@adventure-works.com', Item='Road-650 Black, 62', Quantity=1, UnitPrice=699.0982, TaxAmount=55.9279), Row(SalesOrderNumber='SO43702', SalesOrderLineNumber=1, OrderDate=datetime.date(2019, 7, 1), CustomerName='Colin Anand', EmailAddress='colin45@adventure-works.com', Item='Road-150 Red, 44', Quantity=1, UnitPrice=3578.27, TaxAmount=286.2616), Row(SalesOrderNumber='SO43714', SalesOrderLineNumber=1, OrderDate=datetime.date(2019, 7, 3), CustomerName='Latasha Alonso', EmailAddress='latasha8@adventure-works.com', Item='Road-150 Red, 44', Quantity=1, UnitPrice=3578.27, TaxAmount=286.2616), Row(SalesOrderNumber='SO43726', SalesOrderLineNumber=1, OrderDate=datetime.date(2019, 7, 4), CustomerName='Kelvin Huang', EmailAddress='kelvin25@adventure-works.com', Item='Road-650 Black, 58', Quantity=1, UnitPrice=699.0982, TaxAmount=55.9279), Row(SalesOrderNumber